<a href="https://colab.research.google.com/github/Dicere/Machine_Learning_algorithms_from_scratch/blob/main/Linear_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import load_diabetes

data = load_diabetes(as_frame=True)
X, y = data['data'], data['target']

In [3]:
X

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641
...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,0.044485
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930


In [117]:
class MyLineReg():
    def __init__(self, n_iter=100, learning_rate=0.1,weights=None,metric=None,verbose=1,score = 0,reg = None,l1_coef=0,l2_coef=0):         
        self.n_iter= n_iter
        self.learning_rate = learning_rate
        self.weights = weights
        self.verbose = verbose
        self.metric = metric
        self.score = score
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef

    def __str__(self):
        attributes = ', '.join(f"{key}={value}" for key, value in vars(self).items())
        return f"MyLineReg class: {attributes}"

    def __repr__(self):
        attributes = ', '.join(f"{key}={value}" for key, value in vars(self).items())
        return f"MyLineReg class: {attributes}"

    def fit(self,X_fit:pd.DataFrame,y_fit:pd.Series,n_iter=None,learning_rate=None,verbose=None):
      X = X_fit.copy()
      y = y_fit.copy()
      X.insert(0, "One", 1)
      n = X.shape[0]
      self.weights =  np.ones(X.shape[1])

      def metrics(n,y,y_pred,metrics):
          if metrics=="mae":
            return np.mean(np.abs(y - y_pred))
          if metrics=="mse":
            return np.mean((y - y_pred)**2)
          if metrics=="rmse":
            return np.sqrt(np.mean((y - y_pred) ** 2))
          if metrics=="r2":
            mean_y = np.mean(y)
            ss_total = np.sum((y - mean_y)**2)
            ss_residual = np.sum((y - y_pred)**2)
            r2 = 1 - (ss_residual / ss_total)
            return r2
          if metrics=="mape":
            return (100/n*sum(abs((y-y_pred)/y)))

      def regularization(self):
          if self.reg =="l1":
            return self.l1_coef*np.sign(self.weights)
          elif self.reg=="l2": 
            return self.l2_coef*2*self.weights
          elif self.reg=="elasticnet":
            return self.l1_coef*np.sign(self.weights)+self.l2_coef*2*self.weights
          else :
            return 0

      if n_iter is not None:
        self.n_iter = n_iter
      if learning_rate is not None:
        self.learning_rate = learning_rate
      if verbose is not None:
        self.verbose = verbose

      for i in range(self.n_iter):
        
        y_pred_vec = X.to_numpy()@self.weights
        error = (y_pred_vec - y)
        grad_vec = (2/n)*(error@(X.to_numpy()))+regularization(self)

        self.weights -= self.learning_rate*grad_vec
        self.score = metrics(n,y,y_pred_vec,self.metric)
        if i !=0 and self.verbose != 0:
          if i % self.verbose == 0:
            print(f'{i}:{((y_pred_vec - y)**2).sum()}--{self.metric}:{metrics(n,y,y_pred_vec,self.metric)}')

    def get_coef(self):
        return self.weights[1:]

    def predict(self,X_test:pd.DataFrame):
        X = X_test.copy()
        X.insert(0, "One", 1)
        y_pred_vec = X.to_numpy()@self.weights
        return np.sum(y_pred_vec,axis=1)
  
    def get_best_score(self):
        if self.score is not None:
              return float(self.score)
        else:
              return 0



In [115]:
instance = MyLineReg(metric = 'mse',reg=None)

In [116]:
instance.fit(X,y,n_iter=100,learning_rate=0.1,verbose=10)

10:2695164.890381423--mse:6097.658123034893
20:2547427.514073178--mse:5763.410665323932
30:2514457.756384565--mse:5688.818453358745
40:2483801.974524459--mse:5619.461480824568
50:2454129.4644127823--mse:5552.329105006295
60:2425391.893420503--mse:5487.311976064487
70:2397556.8816356026--mse:5424.336836279644
80:2370593.359381061--mse:5363.333392264844
90:2344471.3582203793--mse:5304.233842127555
100:2319161.969806495--mse:5246.972782367636


In [113]:
instance.get_best_score()

5246.972782367636